In [27]:
# !pip install shapely

In [28]:
from models.erfnet_road import ERFNet
import torch
import numpy as np
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from load_dataset import ImageDataset
from train import train_one_epoch, CrossEntropyLoss2d

FileNotFoundError: Could not find module 'C:\Users\vietlq4\Anaconda3\Library\bin\geos_c.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [7]:
pretrained_model = torch.load('./pretrained_models/weights_erfnet_road.pth',  map_location=torch.device('cpu'))
new_mw = {}
for k,w in pretrained_model.items():
    new_mw[k[7:]] = w

In [8]:
net = ERFNet(num_classes=4)

In [9]:
net.state_dict().update(new_mw)

In [10]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
np.random.seed(50)
torch.manual_seed(50)

if torch.cuda.is_available():
    torch.cuda.manual_seed(50)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((360, 640)),
    #torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor()
])
train_dataset = ImageDataset(
    root_dir='training/',
    txt_files='data/train_list.txt',
    img_size=(360,640),
    transform = transform
    )
val_dataset = ImageDataset(
    root_dir='training/',
    txt_files='data/val_list.txt',
    trainsform = transform
    )
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size,
                            shuffle=True, num_workers=4)

AssertionError: No need ToTensor operation because it is done implicitly already.

In [24]:
num_epochs = 50
max_acc = 0
patience = 10
not_improved_count = 0

In [ ]:
criterion = CrossEntropyLoss2d()
optimizer = torch.optim.Adam(net.parameters(),5e-4,(0.9, 0.999), eps=1e-08, weight_decay=1e-4)

for epoch in range(num_epochs):
    _, acc = train_one_epoch(model=net, criterion=criterion, train_loader, val_loader, device, 100)
    if acc > max_acc:
        torch.save(net.state_dict(), '/content/drive/MyDrive/checkpoints/ERF_epoch_' + str(epoch) + '_acc_{0:.4f}'.format(acc)+'.pt')
        max_acc = acc
        not_improved_count = 0
    else:
        not_improved_count+=1
    
    if not_improved_count >=patience:
        break



